In [37]:
import pandas as pd
from string import ascii_lowercase, ascii_uppercase

import gensim
import numpy as np

In [40]:
model = gensim.models.KeyedVectors.load_word2vec_format(
    '../word2vec_model/GoogleNews-vectors-negative300.bin', binary=True) 


In [41]:
def create_char_dicts(non_letter_chars, lower_case=True, upper_case=True):
    """
    Create dictionary mapping characters to indices
    :param non_letter_chars: list of characters which should be supported other than letters
    :param lower_case: Should set of english lowercase letters be included; default True
    :param upper_case: Should set of english uppercase letters be included; default True
    """
    lower_case_letter_dict={}
    upper_case_letter_dict={}
    index_count = 0
    # Create a dictionary with upper and lower case letters and associated index
    # Note: We include underscores, hyphens, and apostrophes but ignore other characters
    # found in word2vec model, including chinese symbols, emojis, etc
    if lower_case:
        lower_case_letter_dict = {letter: int(index)+index_count for index, letter in enumerate(ascii_lowercase, start=1)}
        index_count += 26
    if upper_case:
        upper_case_letter_dict = {letter: int(index)+index_count for index, letter in enumerate(ascii_uppercase, start=1)} 
        index_count += 26
        
    chardict = {**lower_case_letter_dict, **upper_case_letter_dict}
    
    for char in non_letter_chars:
        chardict[char] = index_count
        index_count += 1

    # Creation of reverse character lookup for debugging and word creation
    reverse_chardict = {}
    for k,v in chardict.items():
        reverse_chardict[v] = k
    
    return chardict, reverse_chardict

supported_non_letter_characters = ['-','\'']
chardict, reverse_chardict = create_char_dicts(supported_non_letter_characters)


In [42]:
data = pd.read_csv("quora_train.csv")

In [43]:
data['question1'] = data['question1'].fillna('')
data['question2'] = data['question2'].fillna('')

In [44]:
all_questions = list(data['question1'])+list(data['question2'])

In [45]:
word_dict = {}
chars_to_remove=['?','!','_','(',')','[',']','..','...']
sc = set(chars_to_remove)

for question in all_questions:
    try:
        words = question.split()
        for word in words:
            word = ''.join([c for c in word if c not in sc])
            word = word.replace("/"," ")
            word_dict[word] = 1
    except:
        print(question)


In [ ]:
def predict_word(word):
    pass

In [ ]:
emb_dict_model = {}
emb_dict_none = {}

for k,v in word_dict.items():
    if model.vocab.get(k):
        emb_dict_model[k] = model[k]
        emb_dict_none[k] = model[k]
    else:
        if all(char in chardict.keys() for char in k):
            emb_dict_model[k] = predict_word(k)
            emb_dict_none[k] = avg_embedding
        else:
            emb_dict_model[k] = avg_embedding
            emb_dict_none[k] = avg_embedding
    
    

In [ ]:
# Create embedding layer which uses Google word2vec if known or Mimick if unknown

## Keras sample code for how to create embedding layer from pretrained; will repeat with each embedding dict from above
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [ ]:
# Create embedding layer which uses Google word2vec if known, or overall avg. if unknown

In [ ]:
# Build Q1 LSTM

model_1_input = Input()
embeddings = model_1_input(embedding_layer)
model_1 = LSTM()(embeddings)



In [ ]:
# Build Q2 LSTM

model_2_input = Input()
embeddings = model_2_input(embedding_layer)
model_2 = LSTM()(embeddings)



In [ ]:
# Merge LSTMS and concat.  Go through dense sigmoid to predict same or not same
joined = keras.layers.Concatenate(axis=-1,[model_1,model_2])
output = Dense()(joined)

model = keras.models.Model(inputs=[model_1_input, model_2_input], outputs=output)


In [ ]:
# Compare results